In [2]:
pip install pulp

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
  Obtaining dependency information for pulp from https://files.pythonhosted.org/packages/f5/78/54928786de173be3210963d198e4a7d037f9bf5dd327430090bf6c52e152/pulp-3.1.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 53.3 MB/s eta 0:00:00 0:00:010:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


# Adaptive Forecasting Model

In [5]:
import numpy as np
import pandas as pd

class ForecastingModel:
    def __init__(self):
        pass

    # ------------------------
    # Moving Average Forecast
    # ------------------------
    def moving_average(self, demand_series, window_size):
        """
        Moving Average Forecasting
        Averages demand over a fixed window of previous values.

        Parameters:
        - demand_series: list of demand values
        - window_size: number of periods to average over

        Returns:
        - List of forecasted values
        """
        if len(demand_series) < window_size:
            raise ValueError("Not enough data points for the specified window size.")

        forecasts = []

        # Slide the window across the series and compute the average
        for t in range(len(demand_series) - window_size):
            window = demand_series[t:t + window_size]
            forecasts.append(np.mean(window))

        return forecasts

    # -------------------------------
    # Exponential Smoothing Forecast
    # -------------------------------
    def exponential_smoothing(self, demand_series, alpha):
        """
        Exponential Smoothing
        Forecast is a weighted average of past actual and past forecast.

        Parameters:
        - demand_series: list of demand values
        - alpha: smoothing factor (between 0 and 1)

        Returns:
        - List of forecasted values
        """
        smoothed_values = [demand_series[0]]  # Initialize with first actual value

        # Recursively apply smoothing formula
        for t in range(1, len(demand_series)):
            last_forecast = smoothed_values[t - 1]
            current_actual = demand_series[t - 1]
            new_forecast = alpha * current_actual + (1 - alpha) * last_forecast
            smoothed_values.append(new_forecast)

        return smoothed_values

    # -----------------------
    # Holt’s Linear Model
    # -----------------------
    def holt_linear(self, demand_series, alpha, beta):
        """
        Holt’s Linear Trend Forecasting
        Accounts for both level and trend in the data.

        Parameters:
        - demand_series: list of demand values
        - alpha: smoothing for level
        - beta: smoothing for trend

        Returns:
        - List of forecasted values
        """
        # Initialize level and trend
        level = demand_series[0]
        trend = demand_series[1] - demand_series[0]

        forecasts = [level]  # First forecast is initial level

        for t in range(1, len(demand_series)):
            last_level = level

            # Update level with trend and observed demand
            level = alpha * demand_series[t] + (1 - alpha) * (level + trend)

            # Update trend based on level difference
            trend = beta * (level - last_level) + (1 - beta) * trend

            # Forecast = level + trend
            forecasts.append(level + trend)

        return forecasts

    # --------------------------
    # Winters’ Additive Model
    # --------------------------
    def winters_additive(self, demand_series, season_length, alpha, beta, gamma):
        """
        Holt-Winters’ Additive Seasonal Forecasting
        Models level, trend, and repeating seasonal effects additively.

        Parameters:
        - demand_series: list of demand values
        - season_length: number of periods per seasonal cycle
        - alpha: level smoothing factor
        - beta: trend smoothing factor
        - gamma: seasonal smoothing factor

        Returns:
        - List of forecasted values
        """
        n = len(demand_series)

        # Require at least 2 seasonal cycles of data for stable initialization
        if n < 2 * season_length:
            raise ValueError("Need at least two full seasonal cycles of data.")

        # Step 1: Initialize level as average of first season
        level = np.mean(demand_series[:season_length])

        # Step 2: Initialize seasonal components
        seasonal_indices = self._initialize_seasonal_components(demand_series, season_length)

        # Step 3: Initialize trend based on change over one season
        trend = np.mean([
            demand_series[i + season_length] - demand_series[i]
            for i in range(season_length)
        ]) / season_length

        forecasts = []

        # Step 4: Recursive smoothing loop
        for t in range(n):
            seasonal_index = t % season_length
            current_actual = demand_series[t]

            if t == 0:
                forecasts.append(current_actual)  # First forecast is the first actual
                continue

            # Save current level for trend update
            last_level = level

            # Update level
            level = alpha * (current_actual - seasonal_indices[seasonal_index]) + (1 - alpha) * (level + trend)

            # Update trend
            trend = beta * (level - last_level) + (1 - beta) * trend

            # Update seasonal component
            seasonal_indices[seasonal_index] = gamma * (current_actual - level) + (1 - gamma) * seasonal_indices[seasonal_index]

            # Forecast = level + trend + seasonal
            forecast = level + trend + seasonal_indices[seasonal_index]
            forecasts.append(forecast)

        return forecasts

    # --------------------------------------
    # Initial Seasonal Components (Additive)
    # --------------------------------------
    def _initialize_seasonal_components(self, data, season_length):
        """
        Estimate initial seasonal components by averaging de-trended values.

        Parameters:
        - data: demand series
        - season_length: number of periods in a season

        Returns:
        - List of seasonal indices
        """
        seasonal_indices = [0] * season_length
        num_full_seasons = len(data) // season_length
        season_averages = []

        # Step 1: Average each season
        for s in range(num_full_seasons):
            start = s * season_length
            end = start + season_length
            avg = np.mean(data[start:end])
            season_averages.append(avg)

        # Step 2: Calculate seasonal index for each position in the cycle
        for i in range(season_length):
            seasonal_sum = 0.0
            for s in range(num_full_seasons):
                seasonal_sum += data[s * season_length + i] - season_averages[s]
            seasonal_indices[i] = seasonal_sum / num_full_seasons

        return seasonal_indices


In [9]:
import pandas as pd

def prompt_parameters(method_name):
    # Ask for parameters based on the selected forecasting method
    if method_name == "moving_average":
        window_size = int(input("Enter window size (e.g., 3): "))
        return {"window_size": window_size}
    
    elif method_name == "exponential_smoothing":
        alpha = float(input("Enter alpha (e.g., 0.3): "))
        return {"alpha": alpha}
    
    elif method_name == "holt_linear":
        alpha = float(input("Enter alpha (e.g., 0.3): "))
        beta = float(input("Enter beta (e.g., 0.1): "))
        return {"alpha": alpha, "beta": beta}
    
    elif method_name == "winters_additive":
        season_length = int(input("Enter season length (e.g., 4): "))
        alpha = float(input("Enter alpha (e.g., 0.3): "))
        beta = float(input("Enter beta (e.g., 0.1): "))
        gamma = float(input("Enter gamma (e.g., 0.1): "))
        return {"season_length": season_length, "alpha": alpha, "beta": beta, "gamma": gamma}
    
    else:
        raise ValueError("Unsupported method.")
        
def run_forecast(method_name, demand_series, **params):
    model = ForecastingModel()

    if not hasattr(model, method_name):
        raise ValueError(f"Method '{method_name}' not found in ForecastingModel.")

    forecast_method = getattr(model, method_name)
    forecasted_values = forecast_method(demand_series, **params)
    return display_forecast_table(method_name.replace("_", " ").title(), demand_series, forecasted_values)
    
def display_forecast_table(method_name, demand_series, forecasted_series):
    # Pads the forecast and creates a table
    forecast_padded = forecasted_series[:]
    while len(forecast_padded) < len(demand_series):
        forecast_padded.insert(0, None)

    df = pd.DataFrame({
        "Period": list(range(1, len(demand_series) + 1)),
        "Actual Demand": demand_series,
        f"{method_name} Forecast": forecast_padded
    })

    print(f"\n {method_name} Forecast Table:\n")
    print(df)

    # Convert None to np.nan for numeric calculation
    forecast_for_metrics = [f if f is not None else np.nan for f in forecast_padded]

    # Compute metrics
    calculate_forecast_metrics(demand_series, forecast_for_metrics, method_name)

    return df
    
def calculate_forecast_metrics(actual, forecast, method_name=""):
    """
    Calculates forecast error metrics: Bias, MAD, MAPE, MSE, Tracking Signal.
    """
    actual = np.array(actual)
    forecast = np.array(forecast)

    # Remove None values (used for alignment)
    valid_idx = ~np.isnan(forecast)
    actual = actual[valid_idx]
    forecast = forecast[valid_idx]

    error = actual - forecast
    abs_error = np.abs(error)
    pct_error = np.abs(error / actual) * 100
    squared_error = error ** 2

    bias = np.mean(error)
    mad = np.mean(abs_error)
    mape = np.mean(pct_error)
    mse = np.mean(squared_error)

    # Tracking signal: cumulative error / MAD
    cumulative_error = np.sum(error)
    ts = cumulative_error / mad if mad != 0 else np.nan

    metrics = {
        "Bias": bias,
        "MAD": mad,
        "MAPE (%)": mape,
        "MSE": mse,
        "Tracking Signal": ts
    }

    print("\n Forecast Accuracy Metrics:")
    for k, v in metrics.items():
        print(f"{k:>16}: {v:.4f}")

    # -----------------------
    # Interpretations / Feedback
    # -----------------------
    print("\n Forecast Feedback:")

    # MAPE feedback
    if mape < 10:
        print("Excellent forecast accuracy (MAPE < 10%).")
    elif mape < 20:
        print("Good forecast accuracy (MAPE < 20%).")
    elif mape < 50:
        print("Acceptable forecast, consider model refinement.")
    else:
        print("Poor forecast accuracy. Consider changing model or parameters.")

    # Bias feedback
    if bias > 5:
        print("Forecast is consistently underestimating demand.")
    elif bias < -5:
        print("Forecast is consistently overestimating demand.")
    else:
        print("Forecast bias is minimal — well centered.")

    # Tracking Signal feedback
    if abs(ts) > 4:
        print("Tracking signal indicates forecast bias. Review needed.")
    else:
        print("Tracking signal is within acceptable control limits.")

    # ---------------------
    # Final Recommendation
    # ---------------------
    print("\n Recommendation:")

    if mape > 50 or abs(ts) > 4:
        print(f"The selected method '{method_name}' does not appear suitable.")
        if method_name == "moving_average":
            print("Try Exponential Smoothing or Holt’s Linear if trend is present.")
        elif method_name == "exponential_smoothing":
            print("Try Holt’s Linear if trend exists, or Winters' model if seasonality is expected.")
        elif method_name == "holt_linear":
            print("Try Winters’ Additive if seasonality exists.")
        elif method_name == "winters_additive":
            print("Consider revisiting seasonal period or using a machine learning-based model.")
    else:
        print(f"The method '{method_name}' is appropriate for this demand pattern.")

    return metrics

def main():
    # Step 1: Input demand series
    raw_input = input("Enter demand values separated by commas (e.g., 120,130,125,...): ")
    demand_series = [float(x.strip()) for x in raw_input.split(",")]

    # Step 2: Choose forecasting method
    print("\nSelect forecasting method:")
    print("1 - Moving Average")
    print("2 - Exponential Smoothing")
    print("3 - Holt Linear")
    print("4 - Winters Additive")

    choice = input("Enter your choice (1-4): ")

    method_map = {
        "1": "moving_average",
        "2": "exponential_smoothing",
        "3": "holt_linear",
        "4": "winters_additive"
    }

    if choice not in method_map:
        print("Invalid choice.")
        return

    method_name = method_map[choice]

    # Step 3: Prompt for required parameters dynamically
    params = prompt_parameters(method_name)

    # Step 4: Run forecast and display
    run_forecast(method_name, demand_series, **params)

if __name__ == "__main__":
    main()

Enter demand values separated by commas (e.g., 120,130,125,...):  120, 131, 122, 140, 135, 150



Select forecasting method:
1 - Moving Average
2 - Exponential Smoothing
3 - Holt Linear
4 - Winters Additive


Enter your choice (1-4):  3
Enter alpha (e.g., 0.3):  0.2
Enter beta (e.g., 0.1):  0.1



 Holt Linear Forecast Table:

   Period  Actual Demand  Holt Linear Forecast
0       1          120.0            120.000000
1       2          131.0            142.000000
2       3          122.0            148.600000
3       4          140.0            157.308000
4       5          135.0            162.828240
5       6          150.0            169.987867

 Forecast Accuracy Metrics:
            Bias: -17.1207
             MAD: 17.1207
        MAPE (%): 12.7503
             MSE: 383.6754
 Tracking Signal: -6.0000

 Forecast Feedback:
Good forecast accuracy (MAPE < 20%).
Forecast is consistently overestimating demand.
Tracking signal indicates forecast bias. Review needed.

 Recommendation:
The selected method 'Holt Linear' does not appear suitable.


# Facility Location Optimization Model

In [7]:
from pulp import PULP_CBC_CMD, LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpStatus
import pandas as pd
import numpy as np

class FacilityLocationOptimizer:
    def __init__(self, fixed_costs, shipping_costs, capacities, demands):
        """
        Initializes the facility location optimizer.

        Args:
        - fixed_costs: list of fixed opening costs for each facility (length m)
        - shipping_costs: 2D list of shipping costs [facility][customer] (shape m x n)
        - capacities: list of capacities for each facility (length m)
        - demands: list of demands for each customer (length n)
        """
        self.fixed_costs = fixed_costs
        self.shipping_costs = shipping_costs
        self.capacities = capacities
        self.demands = demands
        self.num_facilities = len(fixed_costs)
        self.num_customers = len(demands)

    def solve(self, msg = False):
        # Create the problem
        model = LpProblem("Facility_Location", LpMinimize)

        # Decision variables
        y = [LpVariable(f"open_{i}", cat=LpBinary) for i in range(self.num_facilities)]
        x = [[LpVariable(f"x_{i}_{j}", lowBound=0) for j in range(self.num_customers)] 
             for i in range(self.num_facilities)]

        # Objective: Minimize total cost (fixed + shipping)
        model += lpSum(self.fixed_costs[i] * y[i] for i in range(self.num_facilities)) + \
                 lpSum(self.shipping_costs[i][j] * x[i][j] 
                       for i in range(self.num_facilities) 
                       for j in range(self.num_customers))

        # Constraint: Each customer's demand must be met
        for j in range(self.num_customers):
            model += lpSum(x[i][j] for i in range(self.num_facilities)) == self.demands[j], f"demand_{j}"

        # Constraint: Facility capacity cannot be exceeded
        for i in range(self.num_facilities):
            model += lpSum(x[i][j] for j in range(self.num_customers)) <= self.capacities[i] * y[i], f"capacity_{i}"

        # Solve the problem
        solver = PULP_CBC_CMD(msg=msg)
        model.solve(solver)

        # Format the results
        status = LpStatus[model.status]
        facility_open = [y[i].varValue for i in range(self.num_facilities)]
        shipment_plan = [[x[i][j].varValue for j in range(self.num_customers)] for i in range(self.num_facilities)]
        total_cost = model.objective.value()

        return {
            "status": status,
            "total_cost": total_cost,
            "facility_open": facility_open,
            "shipment_plan": shipment_plan
        }

In [8]:
## ----------------------------
# 🔄 Dynamic Input Section
# ----------------------------

def get_input_list(prompt, count, value_type=float):
    return [value_type(input(f"{prompt} {i+1}: ")) for i in range(count)]

def main():
    print("\n Facility Location Optimizer")

    m = int(input("Enter number of facilities: "))
    n = int(input("Enter number of customers: "))

    print("\n Enter fixed costs for each facility:")
    fixed_costs = get_input_list("  Fixed cost for Facility", m)

    print("\n Enter capacities for each facility:")
    capacities = get_input_list("  Capacity for Facility", m)

    print("\n Enter demands for each customer:")
    demands = get_input_list("  Demand for Customer", n)

    print("\n Enter shipping cost from each Facility to each Customer:")
    shipping_costs = []
    for i in range(m):
        print(f"  Facility {i+1}:")
        row = get_input_list(f"    Cost to Customer", n)
        shipping_costs.append(row)

    # Solve the optimization problem
    optimizer = FacilityLocationOptimizer(fixed_costs, shipping_costs, capacities, demands)
    results = optimizer.solve(msg=False)

    # Format output
    df = pd.DataFrame(
        results["shipment_plan"],
        columns=[f"Customer {j+1}" for j in range(n)],
        index=[f"Facility {i+1}" for i in range(m)]
    )

    print("\n Shipment Plan:")
    print(df)

    print("\n Facility Open Status:", results["facility_open"])
    print(" Total Cost:", results["total_cost"])

    #  Additional Comments
    print("\n Optimization Insights:")
    
    # Identify which facilities were opened
    opened = [i for i, open_flag in enumerate(results["facility_open"]) if open_flag > 0.5]
    closed = [i for i, open_flag in enumerate(results["facility_open"]) if open_flag < 0.5]
    
    if opened:
        print(f" Opened Facilities: {', '.join(['Facility ' + str(i+1) for i in opened])}")
    else:
        print(" No facilities were opened — demand may be zero or infeasible.")
    
    if closed:
        print(f"Closed Facilities: {', '.join(['Facility ' + str(i+1) for i in closed])}")
    
    # Compute capacity utilization
    print("\n Capacity Utilization of Open Facilities:")
    for i in opened:
        used_capacity = sum(results["shipment_plan"][i])
        utilization = used_capacity / capacities[i] * 100
        print(f"  - Facility {i+1}: {used_capacity:.1f}/{capacities[i]} units ({utilization:.1f}%)")
        if utilization < 50:
            print("Low utilization — consider consolidating.")
        elif utilization > 95:
            print("Near full capacity — risk of overload.")
    
    # Distribution balance check
    if len(opened) > 1:
        max_used = max(sum(results["shipment_plan"][i]) for i in opened)
        min_used = min(sum(results["shipment_plan"][i]) for i in opened)
        imbalance_ratio = max_used / (min_used + 1e-5)
        if imbalance_ratio > 2:
            print("\n Load imbalance detected — one facility handles far more than others.")
            print("Consider redistributing shipments or adjusting shipping costs.")

if __name__ == "__main__":
    main()



 Facility Location Optimizer


Enter number of facilities:  4
Enter number of customers:  2



 Enter fixed costs for each facility:


  Fixed cost for Facility 1:  100
  Fixed cost for Facility 2:  200
  Fixed cost for Facility 3:  150
  Fixed cost for Facility 4:  90



 Enter capacities for each facility:


...

  Facility 3:


    Cost to Customer 1:  120
    Cost to Customer 2:  122


  Facility 4:


    Cost to Customer 1:  131
    Cost to Customer 2:  130



 Shipment Plan:
            Customer 1  Customer 2
Facility 1         0.0         0.0
Facility 2        79.0         0.0
Facility 3        21.0        69.0
Facility 4         0.0       131.0

 Facility Open Status: [0.0, 1.0, 1.0, 1.0]
 Total Cost: 38678.0

 Optimization Insights:
 Opened Facilities: Facility 2, Facility 3, Facility 4
Closed Facilities: Facility 1

 Capacity Utilization of Open Facilities:
  - Facility 2: 79.0/120.0 units (65.8%)
  - Facility 3: 90.0/90.0 units (100.0%)
Near full capacity — risk of overload.
  - Facility 4: 131.0/131.0 units (100.0%)
Near full capacity — risk of overload.
